<a href="https://colab.research.google.com/github/VDidyk/AI/blob/master/module6/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_NLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання 1
Проведіть токенизацію тексту:
 * Розділіть текст на речення
 * Кожне речення розділіть на слова. Результатом має бути `list[list]` де скажімо `data[5][10]` означає 10-те слово з 5-го речення(індексація з 0)
 * Видаліть пунктуацію та стоп-слова
 * Створіть список усіх слів з тексту 'words`

In [1]:
text = """Mrs Pig was very tired: 'Oh dear,' she said to her three little pigs, 'I can’t do this work anymore, I’m afraid you must leave home and make your own way in the world.' So the three little pigs set off.

The first little pig met a man carrying a bundle of straw.

'Excuse me,' said the first little pig politely. 'Would you please sell some of your straw so I can make a house?'

The man readily agreed and the first little pig went off to find a good place to build his house.

The other little pigs carried on along the road and, soon, they met a man carrying a bundle of sticks.

'Excuse me,' said the little pig politely. 'Would you please sell me some sticks so I can build a house?'

The man readily agreed and the little pig said goodbye to his brother.

The third little pig didn’t think much of their ideas:

'I’m going to build myself a much bigger, better, stronger house,' he thought, and he carried off down the road until he met a man with a cart load of bricks.

'Excuse me,' said the third little pig, as politely as his mother had taught him. 'Please can you sell me some bricks so I can build a house?'

'Of course,' said the man. 'Where would you like me to unload them?'

The third little pig looked around and saw a nice patch of ground under a tree.

'Over there,' he pointed.

They all set to work and by nighttime the house of straw and the house of sticks were built but the house of bricks was only just beginning to rise above the ground. The first and second little pigs laughed, they thought their brother was really silly having to work so hard when they had finished."""

In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
sentences = sent_tokenize(text)

data = []

all_words = []

stop_words = set(stopwords.words('english'))

for sentence in sentences:
    words = word_tokenize(sentence)
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    data.append(words)
    all_words.extend(words)

data, all_words[:10]

([['mrs',
   'pig',
   'tired',
   'dear',
   'said',
   'three',
   'little',
   'pigs',
   'work',
   'anymore',
   'afraid',
   'must',
   'leave',
   'home',
   'make',
   'way',
   'world'],
  ['three', 'little', 'pigs', 'set'],
  ['first', 'little', 'pig', 'met', 'man', 'carrying', 'bundle', 'straw'],
  ['said', 'first', 'little', 'pig', 'politely'],
  ['please', 'sell', 'straw', 'make', 'house'],
  ['man',
   'readily',
   'agreed',
   'first',
   'little',
   'pig',
   'went',
   'find',
   'good',
   'place',
   'build',
   'house'],
  ['little',
   'pigs',
   'carried',
   'along',
   'road',
   'soon',
   'met',
   'man',
   'carrying',
   'bundle',
   'sticks'],
  ['said', 'little', 'pig', 'politely'],
  ['please', 'sell', 'sticks', 'build', 'house'],
  ['man', 'readily', 'agreed', 'little', 'pig', 'said', 'goodbye', 'brother'],
  ['third',
   'little',
   'pig',
   'think',
   'much',
   'ideas',
   'going',
   'build',
   'much',
   'bigger',
   'better',
   'stronger',
 

# Завдання 2

 * Зробіть стемінг слів
 * Зробіть лематизацію слів
 * Створіть `pandas.DataFrame` зі словами, що мають різний результат при стемінгу та лематизації. Має бути 3 стовпчика:
  * оригінальне сово
  * результат стемінгу
  * результат лематизації


In [7]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pandas as pd

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


results = []
for word in words:
    stem = stemmer.stem(word)
    lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
    if stem != lemma:
        results.append((word, stem, lemma))

df = pd.DataFrame(results, columns=['Original Word', 'Stem', 'Lemma'])

print(df)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


  Original Word    Stem   Lemma
0        little   littl  little
1        really  realli  really
2         silly   silli   silly


# Завдання 3
Виділіть фрази за такими шаблонами:
  * прикметник + іменник
  * (хоча б 1 прикметник) + іменник
  * (прислівник + дієслово) АБО (дієслово + прислівник)

In [13]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
import re

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_phrases(text, pattern):
    sentences = sent_tokenize(text)
    phrases = []

    for sentence in sentences:
        tokens = word_tokenize(sentence)
        tagged = pos_tag(tokens)

        chunker = nltk.RegexpParser(pattern)
        parsed = chunker.parse(tagged)

        for subtree in parsed.subtrees():
            if subtree.label() == 'PHRASE':
                temp = [w for w, t in subtree.leaves()]
                phrases.append(' '.join(temp))

    return phrases

patterns = [
    "PHRASE: {<JJ>+<NN>}",
    "PHRASE: {<RB>+<VB>|<VB>+<RB>}"
]

for pattern in patterns:
    extracted_phrases = extract_phrases(text,pattern)
    print(f"{extracted_phrases}\n")


['own way', 'first little pig', 'first little pig', 'first little pig', 'good place', 'little pig', 'little pig', 'third little pig', 'cart load', 'third little pig', "'Of course", 'third little pig', 'nice patch']

['’ t', 'work so']



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
